In [8]:
import os
import numpy as np
import torchvision.transforms as transforms
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from PIL import Image
from sklearn.metrics import classification_report as report

In [19]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
train_transformer = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomResizedCrop((224), scale=(0.5, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize
])

val_transformer = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize
])

In [20]:

txt_train_COVID=r'COVID\trainCT_COVID.txt'
txt_train_NonCOVID=r'NonCOVID\trainCT_NonCOVID.txt'


txt_val_COVID=r'COVID\valCT_COVID.txt'
txt_val_NonCOVID=r'NonCOVID\valCT_NonCOVID.txt'


txt_test_COVID=r'COVID\testCT_COVID.txt'
txt_test_NonCOVID=r'NonCOVID\testCT_NonCOVID.txt'


In [21]:
data_COVID = r'C:\Users\PC\ML_Classification_Project\COVID-19-CT-Classification\Data\CT_COVID'
data_NonCOVID = r'C:\Users\PC\ML_Classification_Project\COVID-19-CT-Classification\Data\CT_NonCOVID'

with open(txt_train_COVID, 'r') as file:
    COVID_train_paths = [os.path.join(data_COVID, line) for line in file.read().splitlines()]
    COVID_train_labels = np.ones(len(COVID_train_paths))

with open(txt_train_NonCOVID, 'r') as file:
    NonCOVID_train_paths = [os.path.join(data_NonCOVID, line) for line in file.read().splitlines()]
    NonCOVID_train_labels = np.zeros(len(NonCOVID_train_paths))

with open(txt_val_COVID, 'r') as file:
    COVID_val_paths = [os.path.join(data_COVID, line) for line in file.read().splitlines()]
    COVID_val_labels = np.ones(len(COVID_val_paths))

with open(txt_val_NonCOVID, 'r') as file:
    NonCOVID_val_paths = [os.path.join(data_NonCOVID, line) for line in file.read().splitlines()]
    NonCOVID_val_labels = np.zeros(len(NonCOVID_val_paths))

with open(txt_test_COVID, 'r') as file:
    COVID_test_paths = [os.path.join(data_COVID, line) for line in file.read().splitlines()]
    COVID_test_labels = np.ones(len(COVID_test_paths))

with open(txt_test_NonCOVID, 'r') as file:
    NonCOVID_test_paths = [os.path.join(data_NonCOVID, line) for line in file.read().splitlines()]
    NonCOVID_test_labels = np.zeros(len(NonCOVID_test_paths))


In [22]:
# Load and preprocess the images
train_paths = np.concatenate((COVID_train_paths, NonCOVID_train_paths))
val_paths = np.concatenate((COVID_val_paths, NonCOVID_val_paths))
test_paths = np.concatenate((COVID_test_paths, NonCOVID_test_paths))

train_labels = np.concatenate((COVID_train_labels, NonCOVID_train_labels))
val_labels = np.concatenate((COVID_val_labels, NonCOVID_val_labels))
test_labels = np.concatenate((COVID_test_labels, NonCOVID_test_labels))

train_images = []
for path in train_paths:
    image = Image.open(path).convert("RGB")
    image = train_transformer(image)  # Apply transformations
    train_images.append(image)

# Resize images to a consistent size
resized_train_images = [transforms.Resize((224, 224))(image) for image in train_images]

val_images = []
for path in val_paths:
    image = Image.open(path).convert("RGB")
    image = val_transformer(image)  # Apply transformations
    val_images.append(image)

# Resize images to a consistent size
resized_val_images = [transforms.Resize((224, 224))(image) for image in val_images]

test_images = []
for path in test_paths:
    image = Image.open(path).convert("RGB")
    image = val_transformer(image)  # Apply transformations
    test_images.append(image)

# Resize images to a consistent size
resized_test_images = [transforms.Resize((224, 224))(image) for image in test_images]

val_images = np.array([val_transformer(Image.open(path).convert("RGB")) for path in val_paths])
test_images = np.array([val_transformer(Image.open(path).convert("RGB")) for path in test_paths])

# Perform PCA dimensionality reduction
pca = PCA(n_components=173)  # Choose the desired number of components
train_images_pca = pca.fit_transform(resized_train_images.reshape(resized_train_images.shape[0], -1))
val_images_pca = pca.transform(val_images.reshape(val_images.shape[0], -1))
test_images_pca = pca.transform(test_images.reshape(test_images.shape[0], -1))

# Train the SVM classifier
svm_classifier = SVC()
svm_classifier.fit(train_images_pca, train_labels)

# Evaluate on the validation set
val_pred = svm_classifier.predict(val_images_pca)
val_accuracy = accuracy_score(val_labels, val_pred)

# Evaluate on the test set
test_pred = svm_classifier.predict(test_images_pca)
test_accuracy = accuracy_score(test_labels, test_pred)

print("Validation Accuracy:", val_accuracy)
print("Test Accuracy:", test_accuracy)

C:\Users\PC\AppData\Local\Temp\ipykernel_13508\2796180742.py:19: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  val_images = np.array([val_transformer(Image.open(path).convert("RGB")) for path in val_paths])


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (118,) + inhomogeneous part.